This notebook is was used to figure out how to solve the portfolio optimization problem using a LP/MIP/QP solver. This notebook contains a prototype that attempts to use [CVXPY](https://www.cvxpy.org) to solve the problem.

The problem requires support for:
* Quadratic programming (QP) - supports minimizing a quadratic objective function (i.e., sum of the squared difference)
* Mixed-integer programming - supports integer variables (i.e., number of funds)
* Linear constraints - support constraining overall allocation to 100% and the allocation to any specific asset class or fund to be less than 100%

Because of these requirements, the solver needs to support mixed-integer quadratic programming (MIQP).

The CVXPY library only supports MIQP when it is used in conjunction with a commercial solver such as CPLEX.

In [2]:
# import required packaages
import pandas as pd
impirt cxvpy as cp
import numpy as np

In [3]:
# load the data
from collections import defaultdict
default_types = defaultdict(lambda: float, Ticker="str")
file_path = "data/exposure_matrix.csv"
data = pd.read_csv(file_path, dtype=default_types)
data

,Ticker,Cash,Intl Bonds,US Bonds,Developed,Emerging,Large Cap Value,Large Cap Core,Large Cap Growth,Mid Cap Value,Mid Cap Core,Mid Cap Growth,Small Cap Value,Small Cap Core,Small Cap Growth,REITs,Unclassified
0,BNDX,0.0192,0.9496,0.0291,0.00000,0.0000,0.0001,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0020
1,BSV,0.0114,0.0984,0.8901,0.00000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0001
2,VEA,0.0104,0.0000,0.0000,0.97430,0.0059,0.0018,0.0020,0.0017,0.000400,0.000500,0.000400,0.000100,0.000100,0.000100,0.0002,0.0021
3,VWO,0.0377,0.0000,0.0000,0.22680,0.7340,0.0002,0.0004,0.0004,0.000100,0.000100,0.000100,0.000000,0.000000,0.000000,0.0000,0.0002
4,VTV,-0.0019,0.0000,0.0000,0.00820,0.0000,0.4454,0.2128,0.0335,0.117300,0.138500,0.013200,0.000300,0.000100,0.000000,0.0326,0.0000
5,VV,0.0008,0.0000,0.0000,0.00490,0.0000,0.2150,0.3793,0.1978,0.055400,0.082600,0.041500,0.000100,0.000000,0.000000,0.0226,0.0000
6,VUG,0.0013,0.0000,0.0000,0.00250,0.0000,0.0059,0.5277,0.3257,0.004300,0.039500,0.077500,0.000000,0.000000,0.000100,0.0155,0.0000
7,VOE,0.0020,0.0000,0.0000,0.01260,0.0000,0.0064,0.0229,0.0061,0.391100,0.434000,0.040700,0.001300,0.000000,0.000000,0.0829,0.0000
8,VO,0.0027,0.0000,0.0000,0.01550,0.0000,0.0058,0.0589,0.0399,0.227100,0.371500,0.200800,0.000700,0.000300,0.000000,0.0768,0.0000
9,VOT,0.0024,0.0000,0.0000,0.01778,0.0000,0.0051,0.1025,0.0808,0.031700,0.294200,0.395300,0.000000,0.000600,0.000100,0.0695,0.0000


In [4]:
# Extract fund_matrix (all rows except the footer and first column)
fund_matrix = data.iloc[:-1, 1:].values
fund_matrix

array([[ 1.920e-02,  9.496e-01,  2.910e-02,  0.000e+00,  0.000e+00,
         1.000e-04,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         2.000e-03],
       [ 1.140e-02,  9.840e-02,  8.901e-01,  0.000e+00,  0.000e+00,
         0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         1.000e-04],
       [ 1.040e-02,  0.000e+00,  0.000e+00,  9.743e-01,  5.900e-03,
         1.800e-03,  2.000e-03,  1.700e-03,  4.000e-04,  5.000e-04,
         4.000e-04,  1.000e-04,  1.000e-04,  1.000e-04,  2.000e-04,
         2.100e-03],
       [ 3.770e-02,  0.000e+00,  0.000e+00,  2.268e-01,  7.340e-01,
         2.000e-04,  4.000e-04,  4.000e-04,  1.000e-04,  1.000e-04,
         1.000e-04,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         2.000e-04],
       [-1.900e-03,  0.000e+00,  0.000e+00,  8.200e-03,  0.000e+00,
         4.454e-01,  2.128e-01, 

In [5]:
# Extract desired_allocations (footer row, excluding the first column)
desired_allocations = data.iloc[-1, 1:].values
desired_allocations

array([np.float64(0.0), np.float64(0.0), np.float64(0.2),
       np.float64(0.128), np.float64(0.042), np.float64(0.1),
       np.float64(0.19), np.float64(0.13), np.float64(0.04666666667),
       np.float64(0.04666666667), np.float64(0.04666666667),
       np.float64(0.02333333333), np.float64(0.02333333333),
       np.float64(0.02333333333), np.float64(0.0), np.float64(0.0)],
      dtype=object)

In [6]:
# Extract fund tickers (first column, excluding the footer row)
fund_tickers = data.iloc[:-1, 0].values
fund_tickers

array(['BNDX ', 'BSV ', 'VEA', 'VWO', 'VTV', 'VV', 'VUG', 'VOE', 'VO',
       'VOT', 'VBR', 'VB', 'VBK'], dtype=object)

In [7]:
# Extract asset classes (header row, excluding the first column)
asset_classes = data.columns[1:]
asset_classes

Index(['Cash', 'Intl Bonds', 'US Bonds', 'Developed', 'Emerging',
       'Large Cap Value', 'Large Cap Core', 'Large Cap Growth',
       'Mid Cap Value', 'Mid Cap Core', 'Mid Cap Growth', 'Small Cap Value',
       'Small Cap Core', 'Small Cap Growth', 'REITs', 'Unclassified'],
      dtype='object')

In [8]:
# Define the optimization problem
num_funds = fund_matrix.shape[0]
x = cp.Variable(num_funds)  # Allocation to each fund

# Resulting portfolio allocation
portfolio_allocation = fund_matrix.T @ x

# Objective: Minimize the squared difference between actual and desired allocations
objective = cp.Minimize(cp.sum_squares(portfolio_allocation - desired_allocations))

# Constraints
constraints = [
    cp.sum(x) == 1,  # Allocations must sum to 100%
    x >= 0           # No negative allocation
]

# Solve the problem
problem = cp.Problem(objective, constraints)
problem.solve()

NameError: name 'cp' is not defined

In [114]:
# Output results
print("Optimal Fund Allocations:")
print(f"{"Ticker":10}{"Allocation":>10}")
print(f"{"========":<10}{"==========":>10}")
for ticker, allocation in zip(fund_tickers, x.value):
    print(f"{ticker:<10}{allocation:10.2%}")

print(type(portfolio_allocation.value))

print("\nResulting Asset Class Allocations:")
print(f"{"Asset Class":20}{"Actual":>10}{"Target":>10}{"Diff":>10}")
for asset_class, actual, target in zip(asset_classes, portfolio_allocation.value, desired_allocations):
    diff = actual - target
    print(f"{asset_class:20}{actual:10.2%}{target:10.2%}{diff:10.2%}")

print("\nObjective Value (total deviation):", problem.value)

Optimal Fund Allocations:
Ticker    Allocation
========  ==========
BNDX          -0.00%
BSV           22.01%
VEA           11.36%
VWO            5.37%
VTV           -0.00%
VV            43.66%
VUG            6.58%
VOE            1.10%
VO            -0.00%
VOT           -0.00%
VBR            1.55%
VB             4.17%
VBK            4.20%
<class 'numpy.ndarray'>

Resulting Asset Class Allocations:
Asset Class             Actual    Target      Diff
Cash                     0.74%     0.00%     0.74%
Intl Bonds               2.17%     0.00%     2.17%
US Bonds                19.59%    20.00%    -0.41%
Developed               12.65%    12.80%    -0.15%
Emerging                 4.02%     4.20%    -0.18%
Large Cap Value          9.48%    10.00%    -0.52%
Large Cap Core          20.09%    19.00%     1.09%
Large Cap Growth        10.81%    13.00%    -2.19%
Mid Cap Value            3.28%     4.67%    -1.38%
Mid Cap Core             5.27%     4.67%     0.61%
Mid Cap Growth           4.05%     4.6